#### <u>On the reason behind the decision</u>

In this notebook, we will develop an algorithm that enables us to elucidate the reason behind a decision made by a classifier represented in a logical form (Conjunctive Normal Form, CNF).<br><br>

The notebook comprises four sections: first, the conversion from Conjunctive Normal Form (CNF) to Deterministic Decomposable Negation Normal Form (d-DNNF); second, an explanation part, specifically the search for sufficient reasons behind a given decision; third, the computation of other interesting queries related to classifiers and decisions; and finally, a case study.<br><br>

The algorithm presented in this notebook is the one described in the article by Adnane Darwich and August Hirth titled "On the Reason Behind the Decision."

In [1]:
import sympy as sp
from sympy.logic import to_cnf

##### <u>1. From Conjunctive Normal Form to dDNNF</u>:
<p>
    In this section, we will build a dDNNF circuit by compiling a Boolean formula in Conjunctive Normal Form (CNF) using the C2D compiler.
</p>

In [2]:
# Definition of some auxiliary functions that allow us to obtain the CNF file.

def count_clauses(formula)-> int:
    """
    Returns the number of clauses of the formula.
    """
    if isinstance(formula, sp.And):
        return len(formula.args)
    elif isinstance(formula, sp.Or):
        return 1
    else:
        return 1


def map_symbols_to_integers(symbols):
    """
    Maps each symbol to a unique integer starting from 1.
    """

    symbol_map = {symbol: i + 1 for i, symbol in enumerate(symbols)}
    return symbol_map

def cnf_to_dimacs(formula, symbol_map):
    """
    Converts a CNF formula to the DIMACS format string using a symbol map.
    """

    clauses = []
    if isinstance(formula, sp.And):
        for clause in formula.args:
            if isinstance(clause, sp.Or):
                clauses.append(' '.join([str(symbol_map[lit]) if not lit.is_Not else f"-{symbol_map[lit.args[0]]}" for lit in clause.args]))
            else:
                clauses.append(str(symbol_map[clause]) if not clause.is_Not else f"-{symbol_map[clause.args[0]]}")

    elif isinstance(formula, sp.Or):
        clauses.append(' '.join([str(symbol_map[lit]) if not lit.is_Not else f"-{symbol_map[lit.args[0]]}" for lit in formula.args]))
        
    else:
        clauses.append(str(symbol_map[formula]) if not formula.is_Not else f"-{symbol_map[formula.args[0]]}")
    
    return clauses

In [3]:
def cnf_to_file(file_name: str, formula, symbol_map)-> None:
    """
    Write a file_name.cnf file corresponding to the conjunctive normal form (CNF) of the formula,
    adhering to the syntax accepted by c2d.
    """
    
    num_vars = len(symbol_map)
    num_clauses = count_clauses(formula)
    clauses = cnf_to_dimacs(formula, symbol_map)
    
    with open(file_name, 'w') as f:

        # This if clause is actually unnecessary, however, since the c2d compiler crashes when given a CNF with a single clause, we included it.
        if( num_clauses == 1 ): 
            f.write(f"p cnf {num_vars} 2\n")
            f.write(f"{clauses[0]} 0\n")
            f.write(f"{clauses[0]} 0\n")

        else:    
            f.write(f"p cnf {num_vars} {num_clauses}\n")
            for clause in clauses:
                f.write(f"{clause} 0\n")

In [4]:
class dDNNF:
    
    children: list[list[int]]
    label: list[str]
    
    def __init__(self, v) -> None:
        self.children = [[] for i in range(v)]
        self.label = ['' for i in range(v)]

In [5]:
def file_to_dDNNF(file: str)-> dDNNF:
    """
    From .cnf.nnf file build the corresponding decision-dnnf tree
    """
    
    with open(file, 'r') as f:

        # Reading the first line
        line = ((f.readline()).strip()).split(' ')
        if(line == []):
            return None
        
        v = int(line[1])
        decision = dDNNF(v)

        line = ((f.readline()).strip()).split(' ')

        i = 0 # Index of nodes
        while(i < v): # Reading the graph's nodes

            if(line[0] == 'A'):
                if(int(line[1]) == 0):
                    decision.label[v - i - 1] = 'T'
                else:
                    for j in range(2, int(line[1]) + 2):
                        decision.children[v - i - 1].append(v - int(line[j]) - 1)
                    decision.label[v - i - 1] = 'A'

            elif(line[0] == 'O'):
                if(int(line[2]) == 0):
                    decision.label[v - i - 1] = 'F'
                else:
                    decision.children[v - i - 1].append(v - int(line[3]) - 1)
                    decision.children[v - i - 1].append(v - int(line[4]) - 1)
                    decision.label[v - i - 1] = 'O' + line[1]

            else:
                decision.label[v - i - 1] = str(line[1])

            i += 1
            line = ((f.readline()).strip()).split(' ')
        
        return decision

In [6]:
def consensus(decision: dDNNF)-> None:
    """
    *** Procedure ***
    Create the consensus circuit obtained from the dDNNF.

    Parameters:
    ----------
    decision (dDNNF): The deterministic decomposable negation normal form, representing the classifier.
    """

    v = len(decision.label)
    for i in range (v):

        if(decision.label[i][0] == 'O'):
            x = int(decision.label[i][1])
            k, j = decision.children[i][0], decision.children[i][1]
            
            cons = []
            for w in (decision.children[k] + decision.children[j]):
                if (decision.label[w]) not in {str(x), str(-x)}:
                    cons.append(w)

            
            decision.children.append(cons)
            decision.children[i].append(len(decision.label))
            decision.label.append('A')

In [7]:
def filter(decision: dDNNF, a: list[bool])-> None:
    """
    *** Procedure ***
    Filter the concensus circuit by instance a.

    Parameters:
    ----------
    decision (dDNNF): consensus(Δ) where Δ is the deterministic decomposable negation normal form, representing the classifier.
    a (list[bool]): The instance on which the filtering process will be performed.
    """
    
    v = len(decision.children)
    for i in range(v):
        if(decision.label[i][0] not in {'O', 'A', 'T', 'F'}):
            x = int(decision.label[i])
            y = abs(x) - 1
            if( (x > 0 and (not a[y])) or  (x < 0 and a[y]) ):
                decision.label[i] = 'F'

##### <u>2. Explanations</u>:
<p>
    In this section, we will use the dDNNF constructed in the previous code to explain the decisions made by the classifier.
</p>

In [8]:
def prod_cart(s1: list[set[int]], s2: list[set[int]])-> list[set[int]]:
    """
    Returns the Cartesian product of s1 with s2.
    """
    return [x | y for x in s1 for y in s2]

def remove_subsumed(r: list[set[int]])-> list[set[int]]:
    """
    Remove subsumed sets from the collection of sets.
    
    Parameters:
    ----------
    r (list[set[int]]): A set containing tuples, where each tuple represents a set.

    Returns:
    set of tuples: A set containing only the non-subsumed sets.
    """
    res = [s1 for s1 in r if not any(s2 < s1 for s2 in r)]
    return [res[i] for i in range (len(res)) if (res[i] not in res[i + 1: ])]

In [9]:
def Pi(decision: dDNNF, i= 0)-> list[set[int]]:
    """
    Generate the list of the decision's sufficient reasons.

    Parameters:
    ----------
    decision (dDNNF): The Filter(consensus(Δ), a) circuit representing the decision.
    i (int): The index of the node we want to explain, default = 0.
    
    Returns:
    -------
    Explanation: A list of the decision's sufficient reasons.
    """

    r : list
    if (decision.label[i] == 'F'):
        r = []
        
    elif (decision.label[i] == 'T'):
        r = [set()]

    elif (decision.label[i] == 'A'):
        j = decision.children[i][0]
        r = Pi(decision, j)
        for j in range(1, len(decision.children[i])):
            r = prod_cart(r, Pi(decision, decision.children[i][j]))

    elif (decision.label[i][0] == 'O'):
        j = decision.children[i][0]
        r = Pi(decision, j)
        for j in range(1, len(decision.children[i])):
            r = r + Pi(decision, decision.children[i][j])
    
    else:
         r = [{int(decision.label[i])}]
    
    r = remove_subsumed(r)
    
    return r

##### <u>3. Computing Queries</u>:

We can now compute various queries using the previous code, specially:
> **Necessary property.**<br>
> **Necessary Reason.**<br>
> **Because Statments.**<br>
> **Even if, Because Statements.**<br>
> **Decision Bias.**<br>

In [10]:
def negate(decision: dDNNF)-> dDNNF:
    """
    Returns the negation of the given dDNNF (deterministic decomposable negation normal form) circuit.
    
    Parameters:
    ----------
    decision (dDNNF): The dDNNF circuit to be negated.
    """

    v = len(decision.label)
    
    negated_decision: dDNNF = dDNNF(v)

    for i in range (v):

        negated_decision.children[i] = decision.children[i]

        if(decision.label[i][0] == 'T'):
            negated_decision.label[i] = 'F'
        elif(decision.label[i][0] == 'F'):
            negated_decision.label[i] = 'T'

        elif(decision.label[i][0] == 'A'):
            negated_decision.label[i] = 'O'
        elif(decision.label[i][0] == 'O'):
            negated_decision.label[i] = 'A'
        
        else:
            x = int(decision.label[i])
            negated_decision.label[i] = str(-x)
        
    return negated_decision

In [11]:
def is_satisfiable(decision: dDNNF, i= 0)-> bool:
    """
    Returns True if and only if the given dDNNF (deterministic decomposable negation normal form) circuit is satisfiable

    Parameters:
    ----------
    decision (dDNNF): The dDNNF circuit.    
    """

    boolean: bool

    if(decision.label[i][0] == 'F'):
        boolean = False

    elif(decision.label[i][0] == 'A'):
        j = decision.children[i][0]
        boolean = is_satisfiable(decision, j)
        for j in range(1, len(decision.children[i])):
            boolean = boolean and is_satisfiable(decision, decision.children[i][j])

    elif(decision.label[i][0] == 'O'):
        j = decision.children[i][0]
        boolean = is_satisfiable(decision, j)
        for j in range(1, len(decision.children[i])):
            boolean = boolean or is_satisfiable(decision, decision.children[i][j])

    else:
        boolean = True
    
    return boolean

In [12]:
def quantify(decision: dDNNF, a: list[bool])-> dDNNF:
    """
    Returns a dDNNF (deterministic decomposable negation normal form) circuit by replacing all literals of the variables
    specified in 'a' with 1 in the given dDNNF (decision).

    Parameters:
    ----------
    decision (dDNNF): The dDNNF circuit representing the decision.
    a (list[bool]): A list of boolean values where each value indicates whether the corresponding variable in the 
                       dDNNF should be replaced with 1.

    Precondition:
    ------------
    The length of 'a' is the number of variables in the dDNNF.
    """

    v = len(decision.label)
    
    quantified_decision: dDNNF = dDNNF(v)

    for i in range(v):

        quantified_decision.children[i] = decision.children[i]
        quantified_decision.label[i] = decision.label[i]
    
        if(decision.label[i][0] not in {'O', 'A', 'T', 'F'}):
            x = int(decision.label[i])
            y = abs(x) - 1
            if(a[y]):
                quantified_decision.label[i] = 'T'
    
    return quantified_decision

In [13]:
def condition(decision: dDNNF, a: dict[int, bool])-> dDNNF:
    """
    Returns a dDNNF (deterministic decomposable negation normal form) circuit by replacing every literal l in decision
    with True if l is in a and its value is True, and replacing it with False if the negation of l is in a and its value is False.

    Parameters:
    ----------
    decision (dDNNF): The dDNNF circuit to be conditioned.
    a (dict[int, bool]): A dictionary where the keys are variable indices and the values are booleans, indicating whether 
                         the variable is set to True or False.
    """

    v = len(decision.label)

    conditioned_decision: dDNNF = dDNNF(v)

    for i in range(v):

        conditioned_decision.children[i] = decision.children[i]

        if(decision.label[i][0] not in {'O', 'A', 'T', 'F'}):
            x = int(decision.label[i])
            y = abs(x)

            if(y in a):
                if( (x > 0 and (a[y])) or  (x < 0 and not a[y]) ):
                    conditioned_decision.label[i] = 'T'
                else:
                    conditioned_decision.label[i] = 'F'
            else:
                conditioned_decision.label[i] = decision.label[i]
        else:
            conditioned_decision.label[i] = decision.label[i]

    return conditioned_decision

In [14]:
def necessary_property(decision: dDNNF, m: int)-> set[int]:
    """
    Returns the set of literlas which are necessary for decision.
    
    Parameters:
    ----------
    decisionThe Filter(consensus(Δ), a) circuit representing the decision.
    m (int): The number of variables.
    """

    res = set()

    for i in range(1, m + 1):
        dico_pos = {i: False}
        dico_neg = {i: True}

        conditioned_decision_pos = condition(decision, dico_pos)
        conditioned_decision_neg = condition(decision, dico_neg)

        if(not is_satisfiable(conditioned_decision_pos)):
            res.add(i)
        if(not is_satisfiable(conditioned_decision_neg)):
            res.add(-i)
        
    return res

In [15]:
def necessary_Reason(decision: dDNNF, m: int)-> set[int]:
    """
    Returns the necessary reason for the decision if it exists.

    Parameters:
    ----------
    decision (dDNNF): The Filter(consensus(Δ), a) circuit representing the decision.
    m (int): The number of variables.
    """

    necessary_prop = necessary_property(decision, m)
    a = {abs(x): x < 0 for x in necessary_prop}

    conditioned_decision = condition(decision, a)

    return necessary_prop if (not is_satisfiable(conditioned_decision)) else set()

In [16]:
def because_t(decision: dDNNF, t: dict[int, bool])-> bool:
    """
    Decide whether the decision was made because of the given property.

    Parameters:
    ----------
    decision (dDNNF): The Filter(consensus(Δ), a) circuit representing the decision.
    t (dict[int, bool]): Property that may have triggered the decision.

    Returns:
    -------
    bool: True if the decision was made because of the property t, False otherwise.
    """

    # First implication:
    negated_decision = negate(decision)
    conditioned_decision = condition(negated_decision, t)

    if(is_satisfiable(conditioned_decision)):
        return False
    
    # Second implication:
    for (c, v) in t.items():
        a = {c: not v}
        conditioned_decision = condition(decision, a)
        if(is_satisfiable(conditioned_decision)):
            return False
        
    return True

In [17]:
def evenIf_because_t(decision: dDNNF, a: list[int], t: dict[int, bool], p: set[int])-> bool:
    """
    Determine whether the decision Δ(a) would remain valid even if ~p because of t.

    Parameters:
    ----------
    decision (dDNNF): The deterministic decomposable negation normal form circuit.
    a (list[int]): The instance upon which the decision Δ(a) was based.
    t (dict[int, bool]): Properties that may have triggered the decision.
    p (set[int]): Properties for which we want to know if the decision would stick if they were negated.

    Returns:
    -------
    bool: True if the decision Δ(a) would stick even if ~p because of t, False otherwise.
    """

    # Copy the dDNNF:
    v = len(decision.label)
    decision_b = dDNNF(v)
    for i in range(v):
        decision_b.children[i] = decision.children[i]
        decision_b.label[i] = decision.label[i]
    
    # Yielding the instance b:
    b = [a[i] if (i + 1 not in p) else 1 - a[i] for i in range(len(a))]

    # Computing the complete reason for decision Δ(b):
    consensus(decision_b)
    filter(decision_b, b)

    # Checking whether it is equivalent to t:
    return because_t(decision_b, t)
    

In [18]:
def decision_bias(decision: dDNNF, unprotected_vars: list[bool])-> bool:
    """
    Decide whether the decision Δ(a) is biased.

    Parameters:
    ----------
    decision (dDNNF): The Filter(consensus(Δ), a) circuit representing the decision.
    unprotected_vars (list[bool]): A list of boolean values where each value indicates whether the corresponding variable is unprotected.

    Returns:
    -------
    bool: True if the decision is biased, False otherwise.
    """

    quantified_decision = quantify(decision, unprotected_vars)
    negated_quantified_decision = negate(quantified_decision)

    return is_satisfiable(negated_quantified_decision)

##### <u>4. Case study</u>:
In this final section, we will validate our previous code -developped in sections 3, 4 and 5- using a case study proposed by Adnan Darwiche in his article 'On the (Complete) Reasons Behind Decisions.'

In [19]:
# Building the logic classifier (Admission classifier):

dnf_formula = sp.simplify_logic(sp.sympify('(C & ((F & (G | W)) | (~F & R))) | (G & R & W)'))
neg_dnf_formula = sp.simplify_logic(sp.Not(dnf_formula))

In [20]:
# Mapping symbols to integers to write the formula in DIMACS format.

symbols = sorted(dnf_formula.free_symbols, key=lambda x: x.name)
symbol_map = map_symbols_to_integers(symbols)
symbol_map

{C: 1, F: 2, G: 3, R: 4, W: 5}

In [21]:
# Defining the applicants:

scott = [1, 0, 1, 1, 1]
robin = [1, 1, 1, 1, 1]
april = [1, 1, 1, 0, 1]

In [22]:
# Building the deterministic decomposable negation normal form circuits (dDNNF) for the three aplicants.

cnf_to_file('posFormula.cnf', to_cnf(dnf_formula), symbol_map)
cnf_to_file('negFormula.cnf', to_cnf(neg_dnf_formula), symbol_map)

!./tools/c2d_linux -in posFormula.cnf
!./tools/c2d_linux -in negFormula.cnf


c2d compiler version 2.20
Copyright (c) Automated Reasoning Group, UCLA 2004-2005
Licensed only for non-commercial, research and educational use

Loaded cnf: 5 vars 5 clauses (0 eclauses)
0 unit clauses, 4 binary clauses, max clause size: 3
Generating dtree...(hgr 5->5): *.********.***.*.********
Sum Cluster=6, cutset=4, context=6, separator=5 done.
Max Cluster=4, Cutset=2, Context=3, Separator=2, Height=3
Compiling...done.
Cache memory: 0.0 MB / Cache count: 0
NNF memory: 0.0 MB
Learned clauses: 0
Compile Time: 0.000s / Pre-Processing: 0.003s / Post-Processing: 0.003s

0.8% of nodes, and 1.0% of edges are dead.
Saving 16 nodes and 18 edges...done.

Total Time: 0.007s

c2d compiler version 2.20
Copyright (c) Automated Reasoning Group, UCLA 2004-2005
Licensed only for non-commercial, research and educational use

Loaded cnf: 5 vars 24 clauses (-8 eclauses)
0 unit clauses, 0 binary clauses, max clause size: 5
Skipped 8 trivial clauses.

Generating dtree...(hgr 16->12): *..*.............

In [23]:
decision = file_to_dDNNF('posFormula.cnf.nnf')

scott_decision = file_to_dDNNF('posFormula.cnf.nnf')
scott_neg_decision = file_to_dDNNF('negFormula.cnf.nnf')

robin_decision = file_to_dDNNF('posFormula.cnf.nnf')
april_decision = file_to_dDNNF('posFormula.cnf.nnf')

consensus(scott_decision)
consensus(robin_decision)
consensus(april_decision)

filter(scott_decision, scott)
filter(robin_decision, robin)
filter(april_decision, april)

**<u>Scott</u>**:<br><br>
According to Darwiche's article, the decision regarding the applicant Scott is biased. The only protected feature is R. Let's verify if our code confirms this.

In [24]:
decision_bias(scott_decision, [1, 1, 1, 0, 1])

True

We can also verify if our code arrives at the same sufficient reasons as those in the article, namely:
> $(C, G, R); (C, R, W); (C, R, -F); (G, R, W)$<br>
> $(1, 3, 4); (1, 4, 5); (1, 4, -2); (3, 4, 5)$

In [25]:
Pi(scott_decision)

[{-2, 1, 4}, {1, 3, 4}, {1, 4, 5}, {3, 4, 5}]

As stated in the article, if we flip the protected characteristic R to -R, the decision will flip with the complete reason being -F, -R so Scott would be denied admission *because* he is not a first time applicant and does not come from a rich hometown.

In [26]:
scott = [1, 0, 1, 0, 1]
consensus(scott_neg_decision)
filter(scott_neg_decision, scott)

r = Pi(scott_neg_decision)
print(f'The complete reason behind the decision is {r}')

b = because_t(scott_neg_decision, {2: False, 4: False})
print(f'Scott would be denied admission because he is not a first time applicant and does not come from a rich hometown = {b}')

The complete reason behind the decision is [{-4, -2}]
Scott would be denied admission because he is not a first time applicant and does not come from a rich hometown = True


**<u>Robin</u>**:<br><br>
According to Darwiche's article, the decision regarding the applicant Robin is not biased. The only protected feature is R. Let's verify if our code confirms this.

In [27]:
decision_bias(robin_decision, [1, 1, 1, 0, 1])

False

We can also verify if our code arrives at the same sufficient reasons as those in the article, namely:
> $(C, F, G); (C, F, W); (C, R, G); (C, R, W); (G, R, W)$<br>
> $(1, 2, 3); (1, 2, 5); (1, 4, 3); (1, 4, 5); (3, 4, 5)$

In [28]:
Pi(robin_decision)

[{1, 2, 3}, {1, 2, 5}, {1, 3, 4}, {1, 4, 5}, {3, 4, 5}]

**<u>April</u>**:<br><br>
According to Darwiche's article, the decision regarding the applicant April is not biased. The only protected feature is R. Let's verify if our code confirms this.

In [29]:
decision_bias(robin_decision, [1, 1, 1, 0, 1])

False

Moreover, $C, F$ are all the necessary characteristics for this decision (i.e., the necessary property). Let's confirm that.

In [30]:
necessary_property(april_decision, 5)

{1, 2}

As stated in the article, the decision on April would stick *even if* she were not to have work experience $(-W)$ because she passed the entrance exam $(C)$, has a good GPA $(G)$ and is a first time applicant $(F)$.

In [31]:
evenIf_because_t(decision, april, {1: True, 3: True, 2: True}, {5})

True

In [33]:
# Deletion of auxiliary files for the construction of the ddnnf
! rm *.cnf *.nnf

rm: impossible de supprimer '*.cnf': Aucun fichier ou dossier de ce nom
rm: impossible de supprimer '*.nnf': Aucun fichier ou dossier de ce nom
